In [ ]:
!pip install sentencepiece
!pip install tokenizers
!pip install transformers

import json
import urllib
import torch.nn as nn
import json
import torch
import pandas as pd
import numpy as np
import transformers
from transformers import  BertForTokenClassification , BertTokenizer, BertForPreTraining, BertConfig, AdamW 
import matplotlib.pyplot as plt
import time

# from timeit import default_timer as timer

import string 
from pathlib import Path
from torch.utils.data import Dataset , DataLoader
import csv

In [ ]:
!nvidia-smi

In [ ]:
NUM_EPOCHS=15
accumulation_steps=4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME='bert-base-cased'
tokenizer=BertTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
#DOWLOAND  TRAINING DATASET

# load dataset
url='https://raw.githubusercontent.com/MikeDoes/ETH_NLP_Project/main/fin_num_5_train.json'
response = urllib.request.urlopen(url)
unprocessed_traindataset=json.loads(response.read()) 

with open('/content/train_dataset.json', 'w') as f:
  json.dump(unprocessed_traindataset, f)

In [ ]:
#DOWLOAND AND VALIDATION DATASET

url='https://raw.githubusercontent.com/MikeDoes/ETH_NLP_Project/main/fin_num_5_validate.json'
response = urllib.request.urlopen(url)
unprocessed_valdataset=json.loads(response.read())
#todo: check with Michael the split. 285 +40 != 409  

with open('/content/validate_dataset.json', 'w') as f:
    json.dump(unprocessed_valdataset, f)

In [ ]:
# DOWNLOAD TESTDATASET

url='https://raw.githubusercontent.com/MikeDoes/ETH_NLP_Project/main/fin_num_5_test.json'
response = urllib.request.urlopen(url)
unprocessed_test=json.loads(response.read())


with open('/content/test_dataset.json', 'w') as f:
  json.dump(unprocessed_test, f)

In [ ]:
def clean(par):
    """
    args: str
    receives a paragraph and cleans its text
    example: data_rows.iloc[0,0]    
    """
    par
    par = par.lower() #remove capital letters
    par = par.replace("[^a-zA-Z]", " ") #remove non english characters
    # remove pontuation, as the punctuation i also converted into tokens
    for char in par:
        if char in string.punctuation:
            par = par.replace(char,"")
    return par

In [ ]:
def categories_list(data):
  ""
  "receives the data and returns all the labels found"
  ""
  labels =[0]
  for i in range(len(data)):
      numbers = data[i]["entities"]
      for j in range(len(numbers)):
        category = numbers[j]["category"]
        labels.append(category)

  labels_uniq = list(set(labels)) #todo: check before 0 is a token for padding, so probably we can t add as 0 
    
  return labels_uniq

In [ ]:
labels_list = categories_list(unprocessed_traindataset)
LABELS_LIST = categories_list(unprocessed_traindataset) 

In [ ]:
def build_label(par,target,categories,tokenizer=tokenizer):
  """
  Args:
      par: paragraph (original paragraph so we can calculate the position 
      targets: format [{'number': '80', 'label': 'percentage', 'pos': 373}]
      categories: full list of categories
    Outputs: a vector of the same size as the tokenized paragraph, with 0s and the label in the word positioning
  """
  #clean the paragraph
  par_clean=clean(par)
  par_token=tokenizer(par_clean)["input_ids"]
  
  #initiates a vector with the same size as the tokenized paragraph
  y_vector = np.zeros(len(par_token)) 
  
  #if the y (target) has still the original categories and not tokenized
  for i in range(len(target)):
    
    pos_start=target[i]["offset_start"] #get the position of the number
    pos_end=target[i]["offset_end"] #get the position of the number
    
    spl_start = par[0:pos_start+1] #split the original paragraph untill position of wanted number (+1 as the position starts in 1)
    spl_end = par[0:pos_end+1] #split the original paragraph untill position of wanted number

    spl_clean_start = clean(spl_start) #now we clean the splitted paragraph
    spl_clean_end = clean(spl_end) #now we clean the splitted paragraph
    
    # tokenize the splitted and clean paragraph and calculate the length of the tokenized vector
    spl_token_start = tokenizer(spl_clean_start,return_tensors='pt')["input_ids"]
    spl_token_end = tokenizer(spl_clean_end,return_tensors='pt')["input_ids"]

    size_start = len(spl_token_start[0])-2 #-1 to remove token=102 which is the end of the vector -1 because lengths does not count with 0
    size_end = len(spl_token_end[0])-2 #-1 to remove token=102 which is the end of the vector -1 because lengths does not count with 0

    diff = size_end - size_start + 1
    pos = size_start

    #in the equivalent position of the number found, we add the category label, which is given by the position in the list
    if diff == 1:
      y_vector[pos] = np.where(np.array(categories)==target[i]["category"])[0][0]
    else:
      pos=size_start
      for k in range(diff):
        # print('k',k,'pos',pos)
        y_vector[pos] = np.where(np.array(categories)==target[i]["category"])[0][0]
        pos =pos+1
   
  y_vector=torch.from_numpy(y_vector)
  y_vector=y_vector.type(torch.LongTensor)
     
  return y_vector#todo

In [ ]:
def recognise_numbers(par,tokenizer = tokenizer):
  """"
  arg: paragraph, string format
  returns: tensor of size of the tokenized paragraph with 1s where the token numbers are
  deterministic way
  """
  
  par_clean=clean(par)
  par_token=tokenizer(par_clean)["input_ids"]
  recognise_numbers = np.zeros(len(par_token))
  pos = 0
  for char in par:
    if char.isdigit() == True:
      spl = par[0:pos+1]
      spl_clean = clean(spl)
      spl_token = tokenizer(spl_clean,return_tensors='pt')["input_ids"]
      size = len(spl_token[0])-2
      recognise_numbers[size] = 1
    pos =pos+1
  
  recognise_numbers=torch.from_numpy(recognise_numbers)
  recognise_numbers=recognise_numbers.type(torch.LongTensor)
  
  return recognise_numbers

In [ ]:
###BUILDING PD DATAFRAME

def extract_list( path_1 : Path):
  with path_1.open() as json_file:
    data=json.load(json_file)

  labels_list=categories_list(data)
  data_rows=[]
  i=0
  for element in data:
    i+=1
    paragraph=clean(element['paragraph'])
    target=build_label( element['paragraph'] , element['entities'] , labels_list ) #TODO : GLOBAL LABEL LIST

    data_rows.append({
        'input' : paragraph ,
        'target' : target  ,
        'original' : element['paragraph']  ,
        'positions' : element['entities']  ,
    })


  return data_rows
      

train_list=extract_list(Path("/content/train_dataset.json"))
validate_list=extract_list(Path("/content/validate_dataset.json"))
test_list=extract_list(Path("/content/test_dataset.json"))


Token indices sequence length is longer than the specified maximum sequence length for this model (1031 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
class NERDataset(Dataset):
  def __init__(
      self,
      data: list,
      tokenizer: BertTokenizer,
  ):
      self.tokenizer=tokenizer,
      self.data=data

  def __len__(self):
    return len(self.data)

  def __getitem__(self,index:int):

    ##DATA ROW
    data_row=self.data[index]


    ##SOURCE ENCODING
    source_encoding=tokenizer(
    data_row['input'],
    return_tensors="pt",
    max_length=512,
    truncation=True)

    labels=data_row["target"]
    index_mask=recognise_numbers(data_row['original'])
    
    labels=labels[0:512]
    index_mask=index_mask[0:512]

    return dict(
        input_ids=source_encoding["input_ids"],
        labels=labels,
        index_mask=index_mask
    )


In [ ]:
class NERmodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = BertForTokenClassification.from_pretrained('bert-base-uncased',problem_type="multi_label_classification").to(device)
    self.model.config.num_labels=10
    self.model.classifier = nn.Linear(in_features=768, out_features=10, bias=False)
    self.softmax_layer=nn.Softmax(dim=-1)
    self.model.to(device)
    

  def forward_prediction(self , input_ids):
    x=self.model(input_ids=input_ids).logits
    output=self.softmax_layer(x)
    return(output)

  
  def forward_train(self , input_ids ):       
    output=self.model(
      input_ids=input_ids,
    )   
    return output

  def disambiguation_train( self , input_ids , index_mask):
    x=self.model(input_ids=input_ids).logits
    output=torch.index_select(x,1,index_mask)
    return output

  def disambiguation_prediction(self , input_ids , index_mask):
    x=self.model(input_ids=input_ids).logits
    x=torch.index_select(x,1,index_mask)
    output=self.softmax_layer(x)
    return output 

  def configure_optimizers(self):
    return AdamW(self.parameters(),lr=0.00003) 

  def print(self):
    print("MODEL CONFIG:", self.model.config , "MODEL STRUCTURE" , self.model)


In [ ]:
def build_index_mask(input_ids,tokenizer = tokenizer):
  par = tokenizer.decode(input_ids[0])
  cut = par[6:len(par)-6] # need to remove the CLS and SEP added by the decoder [CLS] text [SEP]
  index_mask = recognise_numbers(cut,tokenizer=tokenizer)
  return index_mask.unsqueeze(0)
  

In [ ]:
class storecsv(nn.Module):
  def create(path_1):
    headers = ['Status','Epoch','Loss','Accuracy','Accuracy per class','Macro F1','Precision','Recall']
    # rows = [status,epoch,loss,acc,acc_in_class,f1,prec,rec]
    with open(path_1, 'w') as f:
      write = csv.writer(f,delimiter=',')
      write.writerow(headers)
  
  def update(path_1,status,epoch,loss,acc,acc_in_class,f1,prec,rec):
    row = [status,epoch,loss,acc,acc_in_class,f1,prec,rec]
    with open(path_1, 'a') as f:
      write = csv.writer(f)
      write.writerow(row)


  def create_pred(path_1):
    headers = ['Paragraph','Prediction','True']
    # rows = [status,epoch,loss,acc,acc_in_class,f1,prec,rec]
    with open(path_1, 'w') as f:
      write = csv.writer(f,delimiter=',')
      write.writerow(headers)
  
  def update_pred(path_1,par,pred,true):
    row = [par,pred,true]
    with open(path_1, 'a') as f:
      write = csv.writer(f)
      write.writerow(row)

In [ ]:
def reconstruct(outputs):
  """
  receives an output from model.forward_predict (which has the probabilities)
  outputs a list of same length with the categories reconstructed"""
  out_categories = outputs
  y_categories = []
  listy = list(out_categories.size())
  if len(listy)>1:
    for i in range(listy[1]):
      y_categories.append(labels_list[out_categories[0][i]])
  else:
    y_categories = []
    
  return y_categories

In [ ]:
class Accuracy(nn.Module):
  def reset(self):
    self.num_correct = 0
    self.num_examples = 0
    self.fn = 0 # fn = Predicted H0 True H1 + Predict H1 True H0 ## predicted number / word but true is word / number
    self.fp = 0 # fp = Predict H0 True H0 + C1 # predicted number but wrong class
    self.tn = 0 # tn = Predict H1 True H1 # predicted word and it is word
    self.tp = 0 # tp = Predict H0 True H0 + C0 category where cases are truly classified into the positive class #correctly predict number class
    self.predicted_correctly = dict({1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0})
    self.predicted_total = dict({1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0})
    # H0: Number H1: Word
    # C0: correct Class C1: wrong Class
  
  def update(self,y_pred,y_real):
    y_pred = torch.argmax(y_pred, dim=-1)
    subtraction = torch.subtract(y_pred,y_real)

    self.num_correct += y_pred.shape[1]-torch.count_nonzero(subtraction) 
    self.num_examples += y_pred.shape[1]

    for i in range(y_pred.shape[0]):
      if (((y_real[0][i] != 0) and (y_pred[0][i] == 0)) or ((y_real[0][i] == 0) and (y_pred[0][i] != 0))): 
        self.fn += 1
      elif ((y_real[0][i] != 0) and (y_pred[0][i] != 0) and (y_pred[0][i] != y_real[0][i])):
        self.fp += 1
      elif ((y_real[0][i] == 0) and (y_pred[0][i]) == 0):
        self.tn +=1
      elif ((y_real[0][i] != 0) and (y_pred[0][i]) == y_real[0][i]):
        self.tp +=1
      index = y_pred[0][i] 
      if ((y_pred[0][i] == y_real[0][i]) and (y_pred[0][i] != 0)):
        self.predicted_correctly[index.item()] += 1   
        self.predicted_total[index.item()] += 1
      elif (y_pred[0][i] != 0):
        self.predicted_total[index.item()] += 1
      # print('fn',self.fn,'fp',self.fp,'tn',self.tn,'tp',self.tp)
  def accuracy_global(self):
    return self.num_correct/self.num_examples if self.num_examples else 0

  def accuracy_in_class(self):
    res = [0]
    for class_num in range(1,9):
      per_class = self.predicted_correctly[class_num]/self.predicted_total[class_num] if self.predicted_total[class_num] else -1 #when there is no occurance of the category
      res.append(per_class)
    return res

  def precision(self):
    return self.tp/(self.tp + self.fp) if self.tp + self.fp else 0
    
  def recall(self):
    return self.tp/(self.tp + self.fn) if self.tp + self.fn else 0

  def f1_score(self):
    recall_f1 = self.recall()
    precision_f1 = self.precision()
    return 2*(recall_f1 * precision_f1) / (recall_f1 + precision_f1) if recall_f1 + precision_f1 else 0

In [ ]:
def test_model( test_module , model , model_type):

    loss_function = nn.CrossEntropyLoss()
    test_loss=0
    k=0
    model.eval()
    metric=Accuracy()
    metric.reset()
    storecsv.create_pred(path_test_pred)


    if(model_type== 0): #RECOGNITION & DISAMBIGUATION CASE  
      for batch in test_module:

        input_ids=torch.squeeze(batch["input_ids"])      
        labels=torch.squeeze(batch["labels"])

        input_ids=input_ids.view(1,input_ids.shape[-1])
        labels=labels.view(1,labels.shape[-1])

        input_ids=input_ids.to(device)
        labels=labels.to(device)

        output=model.forward_train(input_ids).logits
        probability_output=model.forward_prediction(input_ids)

        loss_output=output.transpose(1,2)    
        loss=loss_function(loss_output, labels)
        test_loss+=loss.item()

        par = tokenizer.decode(input_ids[0])

        pred = reconstruct(torch.argmax(output, dim = -1))
        true = reconstruct(labels)
        storecsv.update_pred(path_test_pred,par[6:-6],pred,true)
        metric.update(probability_output,labels)
        
        k=k+1

      acc = metric.accuracy_global()
      acc_in_class = metric.accuracy_in_class()
      prec = metric.precision()
      rec = metric.recall()
      f1 = metric.f1_score()
      test_loss = test_loss/k
      print(f"average loss --:{test_loss}", f"accuracy --:{acc}",f"accuracy class --:{acc_in_class}",f"f1_score --:{f1}")


    if(model_type== 1 ): #DISAMBIGUATION CASE
      for batch in test_module:

        input_ids=torch.squeeze(batch["input_ids"])      
        labels=torch.squeeze(batch["labels"])
        index_mask=batch["index_mask"]
        

        input_ids=input_ids.view(1,input_ids.shape[-1])
        labels=labels.view(1,labels.shape[-1])

        input_ids=input_ids.to(device)
        labels=labels.to(device)
        index_mask=index_mask.to(device)

        a , index_mask=torch.nonzero(index_mask, as_tuple=True)
        labels=torch.index_select(labels,1,index_mask)

        output=model.disambiguation_train(input_ids, index_mask=index_mask)
        probability_output=model.disambiguation_prediction(input_ids , index_mask=index_mask)
        loss_output=output.transpose(1,2)    
        loss=loss_function(loss_output, labels)
        test_loss+=loss.item()
        metric.update(probability_output,labels)
        par = tokenizer.decode(input_ids[0])
        pred = reconstruct(torch.argmax(output, dim = -1))
        true = reconstruct(labels)
        storecsv.update_pred(path_test_pred,par[6:-6],pred,true)
        
        k=k+1
      
      acc = metric.accuracy_global()

      acc_in_class = metric.accuracy_in_class()
      prec = metric.precision())

      rec = metric.recall()

      f1 = metric.f1_score()

      test_loss=test_loss/k
      print(f"test loss: {test_loss} ",f"accuracy: {acc} ")

    storecsv.update(path_test,'test',"",test_loss, acc, acc_in_class, f1, prec, rec)




In [ ]:
def disambiguation_training(data_module):
  model=NERmodel()
  model=model.to(device)
  optimizer=model.configure_optimizers()
  metric = Accuracy()
  loss_function = nn.CrossEntropyLoss()

  storecsv.create(path_training)
  storecsv.create(path_validation)

  training_loss=[]
  _validation_loss=[]
  training_acc=[]
  validation_acc=[]

  training_acc_in_class=[]
  validation_acc_in_class=[]

  training_prec=[]
  validation_prec=[]

  training_rec=[]
  validation_rec=[]

  training_f1=[]
  validation_f1=[]

  training_acc_in_class=[]
  validation_acc_in_class=[]

  t0=time.time()
  optimizer.zero_grad()
  
  for i in range (NUM_EPOCHS):
    print(f"epoch--{i}")
    loss_count=0
    metric.reset()
    validation_loss=0
    k=0
    model.train()
    for batch in data_module:
      input_ids=torch.squeeze(batch["input_ids"])      
      labels=torch.squeeze(batch["labels"])
      index_mask=batch["index_mask"]

      input_ids=input_ids.view(1,input_ids.shape[-1])
      labels=labels.view(1,labels.shape[-1])

      index_mask=index_mask.to(device)
      input_ids=input_ids.to(device)
      labels=labels.to(device)

      _a , index_mask=torch.nonzero(index_mask, as_tuple=True)

      labels=torch.index_select(labels,1,index_mask).to(device)      

      output=model.disambiguation_train(input_ids, index_mask=index_mask)
      probability_output=model.disambiguation_prediction(input_ids , index_mask=index_mask)

      loss_output=output.transpose(1,2)    
      loss=loss_function(loss_output, labels)
      loss_count+=loss.item()
      loss=loss/accumulation_steps
      loss.backward()
      k+=1

      metric.update(probability_output,labels)

      if(k % accumulation_steps == 0):
        optimizer.step()
        optimizer.zero_grad()

      with torch.no_grad():

        if(k % 20 ==0):
          mean_loss=loss_count/20
          training_loss.append(mean_loss)
          loss_count=0

          acc = metric.accuracy_global()
          training_acc.append(acc)

          acc_in_class = metric.accuracy_in_class()
          training_acc_in_class.append(acc_in_class)

          prec = metric.precision()
          training_prec.append(prec)

          rec = metric.recall()
          training_rec.append(rec)

          f1 = metric.f1_score()
          training_f1.append(f1)

          metric.reset()
          print(f"average loss --:{mean_loss}", f"accuracy --:{acc}", f"f1_score --:{f1}",f"accuracy class --:{acc_in_class}")

    storecsv.update(path_training,'training',i,mean_loss,acc,acc_in_class,f1,prec,rec)
    k=0
    model.eval()
    for batch in validate_module:

      input_ids=torch.squeeze(batch["input_ids"])      
      labels=torch.squeeze(batch["labels"])
      index_mask=batch["index_mask"]
      

      input_ids=input_ids.view(1,input_ids.shape[-1])
      labels=labels.view(1,labels.shape[-1])

      input_ids=input_ids.to(device)
      labels=labels.to(device)
      index_mask=index_mask.to(device)

      a , index_mask=torch.nonzero(index_mask, as_tuple=True)
      labels=torch.index_select(labels,1,index_mask)

      output=model.disambiguation_train(input_ids, index_mask=index_mask)
      probability_output=model.disambiguation_prediction(input_ids , index_mask=index_mask)

      loss_output=output.transpose(1,2)    
      loss=loss_function(loss_output, labels)
      validation_loss+=loss.item()
      metric.update(probability_output,labels)
      
      k=k+1
    
    acc = metric.accuracy_global()
    validation_acc.append(acc)

    acc_in_class = metric.accuracy_in_class()
    validation_acc_in_class.append(acc_in_class)

    prec = metric.precision()
    validation_prec.append(prec)

    rec = metric.recall()
    validation_rec.append(rec)

    f1 = metric.f1_score()
    validation_f1.append(f1)
    
    storecsv.update(path_validation,'validation',i,validation_loss,validation_acc,validation_acc_in_class,validation_f1,validation_prec,validation_rec)

    metric.reset()

    print(f"validation loss: {validation_loss/k} ",f"accuracy: {acc}", f"accuracy/class: {acc_in_class}", f"f1 score: {f1}")

  training_time=time.time()-t0
  storecsv.update(path_training,'TIME TO TRAIN',training_time,mean_loss,acc,acc_in_class,f1,prec,rec)
  print("training time:", training_time)
  return model

In [ ]:
def recognition_training(data_module):
  model=NERmodel()
  model=model.to(device)
  optimizer=model.configure_optimizers()
  metric = Accuracy()
  loss_function = nn.CrossEntropyLoss()

  storecsv.create(path_training)
  storecsv.create(path_validation)

  training_loss=[]
  _validation_loss=[]
  training_acc=[]
  validation_acc=[]

  training_acc_in_class=[]
  validation_acc_in_class=[]

  training_prec=[]
  validation_prec=[]

  training_rec=[]
  validation_rec=[]

  training_f1=[]
  validation_f1=[]

  training_acc_in_class=[]
  validation_acc_in_class=[]

  optimizer.zero_grad()
  t0=time.time()

  for i in range (NUM_EPOCHS):
    print(f"epoch--{i}")
    loss_count=0
    metric.reset()
    validation_loss=0
    model.train()
    k=0

    for batch in data_module:

      input_ids=torch.squeeze(batch["input_ids"])      
      labels=torch.squeeze(batch["labels"])

      input_ids=input_ids.view(1,input_ids.shape[-1])
      labels=labels.view(1,labels.shape[-1])

      input_ids=input_ids.to(device)
      labels=labels.to(device)

      output=model.forward_train(input_ids).logits
      probability_output=model.forward_prediction(input_ids)


      loss_output=output.transpose(1,2)    
      loss=loss_function(loss_output, labels)
      loss_count+=loss.item()
      loss=loss/accumulation_steps
      loss.backward()
      k+=1

      metric.update(probability_output,labels)

      if(k % accumulation_steps == 0):
        optimizer.step()
        optimizer.zero_grad()

      with torch.no_grad():

        if(k % 20 == 0): # TO DO CHECK WITH CLEMENTE IF 20 SHOULD BE REPLACED BY ACCUMULATION STEP
          mean_loss=loss_count/20
          training_loss.append(mean_loss)
          loss_count=0

          acc = metric.accuracy_global()
          training_acc.append(acc)

          acc_in_class = metric.accuracy_in_class()
          training_acc_in_class.append(acc_in_class)

          prec = metric.precision()
          training_prec.append(prec)

          rec = metric.recall()
          training_rec.append(rec)

          f1 = metric.f1_score()
          training_f1.append(f1)
          metric.reset()


          print(f"average loss --:{mean_loss}", f"accuracy --:{acc}" ,f"accuracy class --:{acc_in_class}",f"f1_score --:{f1}")

          
    
    k=0
    storecsv.update(path_training,'training',i,mean_loss,acc,acc_in_class,f1,prec,rec)

    model.eval()
    for batch in validate_module:

      input_ids=torch.squeeze(batch["input_ids"])      
      labels=torch.squeeze(batch["labels"])

      input_ids=input_ids.view(1,input_ids.shape[-1])
      labels=labels.view(1,labels.shape[-1])

      input_ids=input_ids.to(device)
      labels=labels.to(device)

      output=model.forward_train(input_ids).logits
      probability_output=model.forward_prediction(input_ids)

      loss_output=output.transpose(1,2)    
      loss=loss_function(loss_output, labels)
      validation_loss+=loss.item()

      metric.update(probability_output,labels)
      
      k=k+1

    acc = metric.accuracy_global()
    validation_acc.append(acc)

    prec = metric.precision()
    validation_prec.append(prec)

    rec = metric.recall()
    validation_rec.append(rec)

    f1 = metric.f1_score()
    validation_f1.append(f1)

    acc_in_class = metric.accuracy_in_class()
    validation_acc_in_class.append(acc_in_class)

    metric.reset()

    print(f"validation loss: {validation_loss/k} ",f"accuracy: {acc} ", f"accuracy/class: {acc_in_class} ",f"f1_score --:{f1}")
    storecsv.update(path_validation,'validation',i,mean_loss,acc,acc_in_class,f1,prec,rec)

  trainingtime=time.time()-t0
  storecsv.update(path_training,'TIME TO TRAIN',trainingtime,mean_loss,acc,acc_in_class,f1,prec,rec)
  print("trainingtime:", trainingtime)
  return model

In [ ]:
train_dataset=NERDataset(train_list,tokenizer,)
validate_dataset=NERDataset(validate_list,tokenizer)
test_dataset=NERDataset(test_list, tokenizer)

train_module = DataLoader(train_dataset, batch_size=1,shuffle="True") #droplast="True"
validate_module = DataLoader(validate_dataset, batch_size=1 , shuffle="True" )
test_module= DataLoader(test_dataset , batch_size=1 , shuffle="True" )

path_training ='/content/training_D_15_final.csv' #UPDATE_PATH : training_model_epochs_lr_accumulation_dataset_model_name
path_validation = '/content/vallidation__D_final.csv' #UPDATE_PATH : validation_model_epochs_lr_accumulation_dataset_model_name 
path_test='/content/test_D__final.csv' #UPDATE_PATH : test_model_epochs_lr_accumulation_dataset_model_name
path_test_pred='/content/test_prediction__final_FFFF.csv' #UPDATE_PATH : test_model_epochs_lr_accumulation_dataset_model_name
#model can be D for disambiguation or RD for recognition and disambiguation

# Disambiguation
model=disambiguation_training(train_module)
torch.save(model.state_dict(), '/content/model_D_weights')
test_model( test_module , model , model_type=1) # 1 FOR DISAMBIGUATION , 0 FOR RECOGNITION &DISAMBIGUATION

#Recognition
# model=recognition_training(train_module)
# torch.save(model.state_dict(), '/content/model_RD_weights')
# test_model( test_module , model , model_type=0) # 1 FOR DISAMBIGUATION , 0 FOR RECOGNITION &DISAMBIGUATION